In [46]:
import tensorflow as tf
import importlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
import utils_optim1
import model1
import results_generator
importlib.reload(utils_optim1)
importlib.reload(model1)
importlib.reload(results_generator)
from utils_optim1 import *
from model1 import *
from results_generator import *
tf.test.is_built_with_cuda()

True

In [ ]:
# This code is only needed if the dataset is stored in google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Check if gpu exists
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# This code is only needed if the dataset is stored in google drive and it is 7z compressed
!p7zip -d /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/MyDrive/Thesis/                                         1 file, 263683120 bytes (252 MiB)

Extracting archive: /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z
--
Path = /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z
Type = 7z
Physical Size = 263683120
Headers Size = 344455
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 681 - Sketch-Icon-Dataset/icon/emoji/sad-tear.jpg                                                        1% 952 - Sketch-Icon-Dataset/icon/mathematics/wave-square.jpg

In [13]:
icon_name_category, sketch_name_category = get_icons_and_sketches()
#icon_dictionary, sketch_dictionary = load_icons_sketches_dic(icon_name_category, sketch_name_category)

In [38]:
# I dont split the data into train and test because I want all of the icons to be trained, since the icons dataset will always be the same
icon_categories =  np.unique(icon_name_category[:, 1], return_index=False)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(icon_categories)

# binary encode the classes of icons
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded).astype(np.float32)
icon_categories_dic = {}
for i in range(len(icon_categories)):
    icon_categories_dic[icon_categories[i]] = onehot_encoded[i]

# len_data = len(icon_name_category)
# p_train=0.9
# p_test=0.1
# num_train = int(np.ceil(len_data*p_train))
# num_test = int(np.floor(len_data*p_test))

# icon_name_category = shuffle(icon_name_category)
# icons_Train = icon_name_category[:num_train]
# icons_Test= icon_name_category[-num_test:]

# print(f'We have {len(icons_Train)} samples in the training set.')
# print(f'We have {len(icons_Test)} samples in the test set.')

In [44]:
BATCH_SIZE = 128

icon_dataset = generate_classification_icon_dataset(icon_name_category, icon_categories_dic, BATCH_SIZE)

In [31]:
# Creation of folders for storing stats, weights and the hyperparameters of the model
model_name = "MyNet"
algorithm_name = "Icon Classification"
create_directory(model_name + "/")
now = datetime.now()
date_time_folder = now.strftime("%d-%m-%Y %H-%M-%S")
train_weights_folder = "Train Weights"
create_directory(model_name + "/" + algorithm_name)
current_run_path = model_name + "/" + algorithm_name + "/" + date_time_folder + "/"
create_directory(current_run_path)
train_weights_path = model_name + "/" + algorithm_name + "/" + date_time_folder + "/" + train_weights_folder
create_directory(train_weights_path)
#=====================================================================================

# Hyperparameter initializations

num_epochs = 50
learing_rate = 0.001

# Store the hyperparameters of the model into a file to remember what I used in a specific run
write_classification_hyperparameters_in_file(current_run_path, learing_rate, BATCH_SIZE)

# Initializations of optimizer, loss and network and set the backend (model variables) to float32
optimizer = tf.keras.optimizers.Adam(learing_rate)
tf.keras.backend.set_floatx('float32')
iconClassificationModel = mynet(len(icon_categories))
loss = compute_cross_entropy

# Function which is responsible to calculate the loss and update the grads
def train_step(icons, targets):
    with tf.GradientTape() as tape:
        _, outputs = iconClassificationModel(icons, training = True)   
        tape.watch(outputs)
        current_loss = loss(outputs, targets)
    grads = tape.gradient(current_loss, iconClassificationModel.trainable_variables)
    optimizer.apply_gradients(zip(grads, iconClassificationModel.trainable_variables))
    return current_loss

Successfully created the directory  MyNet/
Successfully created the directory  MyNet/Icon Classification
Successfully created the directory  MyNet/Icon Classification/13-04-2021 13-25-34/
Successfully created the directory  MyNet/Icon Classification/13-04-2021 13-25-34/Train Weights


In [41]:
top_acc = 0
for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    # Training loop throught the dataset
    for icons, targets in icon_dataset:
        loss_value = train_step(icons, targets)
        epoch_loss_avg.update_state(loss_value)
    print("Epoch {:d}: Loss: {:.3f}".format(epoch,epoch_loss_avg.result()))
    acc = 0
    # Test loop through the same datatet
    for icons,targets in icon_dataset:
        _, outputs = iconClassificationModel(icons, training = False)
        correct_prediction = tf.equal(tf.argmax(outputs,1),tf.argmax(targets,1))
        for prediction in correct_prediction:
            if prediction:
                acc = acc + 1
    
    # Storing of model weights if the accuracy is better than the current max accuracy
    if top_acc<acc:
        top_acc = acc
        save_weights(iconClassificationModel, train_weights_path + "/iconClassification")

    # Storing of epoch, loss and accuracy into a file
    write_classification_stats_in_file(current_run_path, epoch, epoch_loss_avg.result(), acc/len(icon_name_category))
    print("Test accuracy is: " + str(acc/len(icon_name_category)))


Epoch 0: Loss: 4.052
Test accuracy is: 0.0500848896434635
Epoch 1: Loss: 2.745
Test accuracy is: 0.10865874363327674
Epoch 2: Loss: 1.978
Test accuracy is: 0.166383701188455
Epoch 3: Loss: 1.376
Test accuracy is: 0.2266553480475382
Epoch 4: Loss: 0.938
Test accuracy is: 0.35738539898132426
Epoch 5: Loss: 0.618
Test accuracy is: 0.366723259762309
Epoch 6: Loss: 0.409
Test accuracy is: 0.2758913412563667
Epoch 7: Loss: 0.267
Test accuracy is: 0.25127334465195245
Epoch 8: Loss: 0.181
Test accuracy is: 0.33446519524617996
Epoch 9: Loss: 0.128
Test accuracy is: 0.24448217317487267
Epoch 10: Loss: 0.089
Test accuracy is: 0.21477079796264856
Epoch 11: Loss: 0.066
Test accuracy is: 0.233446519524618
Epoch 12: Loss: 0.050
Test accuracy is: 0.2071307300509338
Epoch 13: Loss: 0.040
Test accuracy is: 0.21561969439728354
Epoch 14: Loss: 0.033
Test accuracy is: 0.21477079796264856
Epoch 15: Loss: 0.028
Test accuracy is: 0.21901528013582344
Epoch 16: Loss: 0.025
Test accuracy is: 0.22580645161290322


## Sketch Classification

In [45]:
sketch_categories =  np.unique(sketch_name_category[:, 1], return_index=False)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(sketch_categories)

# binary encode the classes of sketches
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded).astype(np.float32)
sketch_categories_dic = {}
for i in range(len(sketch_categories)):
    sketch_categories_dic[sketch_categories[i]] = onehot_encoded[i]

# Split the dataset into train and test with randomness
# or load the train and test dataset which is created as numpy arrays.
# The purpose of the predefined datasets is to have the same datasets for training
# the classification of sketches and the sketch-icon retrieval, so the test set will be the same,
# and unknown for both trainings
random_generation = False
if random_generation:
    len_data = len(sketch_name_category)
    p_train=0.9
    p_test=0.1
    num_train = int(np.ceil(len_data*p_train))
    num_test = int(np.floor(len_data*p_test))

    sketch_name_category = shuffle(sketch_name_category)
    sketches_Train = sketch_name_category[:num_train]
    sketches_Test= sketch_name_category[-num_test:]
else:
    sketches_Train = load_train_set()
    sketches_Train = shuffle(sketches_Train)
    sketches_Test = load_test_set()

print(f'We have {len(sketches_Train)} samples in the training set.')
print(f'We have {len(sketches_Test)} samples in the test set.')

We have 28626 samples in the training set.
We have 3180 samples in the test set.


In [49]:
BATCH_SIZE = 128

sketch_dataset_Train = generate_classification_sketch_dataset(sketches_Train, sketch_categories_dic, BATCH_SIZE)
sketch_dataset_Test = generate_classification_sketch_dataset(sketches_Test, sketch_categories_dic, BATCH_SIZE)

In [48]:
# Creation of folders for storing stats, weights and the hyperparameters of the model
model_name = "MyNet"
algorithm_name = "Sketch Classification"
create_directory(model_name + "/")
now = datetime.now()
date_time_folder = now.strftime("%d-%m-%Y %H-%M-%S")
train_weights_folder = "Train Weights"
create_directory(model_name + "/" + algorithm_name)
current_run_path = model_name + "/" + algorithm_name + "/" + date_time_folder + "/"
create_directory(current_run_path)
train_weights_path = model_name + "/" + algorithm_name + "/" + date_time_folder + "/" + train_weights_folder
create_directory(train_weights_path)
#=====================================================================================

# Hyperparameter initializations
num_epochs = 300
learing_rate = 0.001

# Store the hyperparameters of the model into a file to remember what I used in a specific run
write_classification_hyperparameters_in_file(current_run_path, learing_rate, BATCH_SIZE)

# Initializations of optimizer, loss and network and set the backend (model variables) to float32
optimizer = tf.keras.optimizers.Adam(learing_rate)
tf.keras.backend.set_floatx('float32')
sketchClassificationModel = mynet(len(sketch_categories))
loss = compute_cross_entropy

# Function which is responsible to calculate the loss and update the grads
def train_step(sketches, targets):
    with tf.GradientTape() as tape:
        _, outputs = sketchClassificationModel(sketches, training = True)   
        tape.watch(outputs)
        current_loss = loss(outputs, targets)
    grads = tape.gradient(current_loss, sketchClassificationModel.trainable_variables)
    optimizer.apply_gradients(zip(grads, sketchClassificationModel.trainable_variables))
    return current_loss

Creation of the directory  failedMyNet/
Successfully created the directory  MyNet/Sketch Classification
Successfully created the directory  MyNet/Sketch Classification/13-04-2021 13-50-30/
Successfully created the directory  MyNet/Sketch Classification/13-04-2021 13-50-30/Train Weights


In [50]:

top_acc = 0
for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    
    # Training loop throught the training dataset
    for sketches, targets in sketch_dataset_Train:
        loss_value = train_step(sketches, targets)
        epoch_loss_avg.update_state(loss_value)
    print("Epoch {:d}: Loss: {:.3f}".format(epoch,epoch_loss_avg.result()))
    
    # Test loop through the test datatet
    acc = 0
    for sketches, targets in sketch_dataset_Test:
        _, outputs = sketchClassificationModel(sketches, training = False)
        correct_prediction = tf.equal(tf.argmax(outputs,1),tf.argmax(targets,1))
        for prediction in correct_prediction:
            if prediction:
                acc = acc + 1
    
    # Storing of model weights if the accuracy is better than the current max accuracy
    if top_acc<acc:
        top_acc = acc
        save_weights(sketchClassificationModel, train_weights_path + "/sketchClassification")
    print("Test accuracy is: " + str(acc/len(sketches_Test)))

    # Storing of epoch, loss and accuracy into a file
    write_classification_stats_in_file(current_run_path, epoch, epoch_loss_avg.result(), acc/len(sketches_Test))

    # shuffle the training set every epoch
    sketches_Train = shuffle(sketches_Train)
print("The best accuracy is: " + str(top_acc/len(sketches_Test)))

Epoch 0: Loss: 3.894
Test accuracy is: 0.00880503144654088
Epoch 1: Loss: 3.290
Test accuracy is: 0.008176100628930818
Epoch 2: Loss: 2.605
Test accuracy is: 0.033647798742138364
Epoch 3: Loss: 1.948
Test accuracy is: 0.01761006289308176
Epoch 4: Loss: 1.396
Test accuracy is: 0.017924528301886792
Epoch 5: Loss: 0.976
Test accuracy is: 0.016666666666666666
Epoch 6: Loss: 0.681
Test accuracy is: 0.009119496855345912
Epoch 7: Loss: 0.450
Test accuracy is: 0.0509433962264151
Epoch 8: Loss: 0.290
Test accuracy is: 0.01949685534591195
Epoch 9: Loss: 0.188
Test accuracy is: 0.06855345911949685
Epoch 10: Loss: 0.127
Test accuracy is: 0.03490566037735849
Epoch 11: Loss: 0.094
Test accuracy is: 0.04339622641509434
Epoch 12: Loss: 0.081
Test accuracy is: 0.04119496855345912
Epoch 13: Loss: 0.080
Test accuracy is: 0.022641509433962263
Epoch 14: Loss: 0.116
Test accuracy is: 0.059748427672955975
Epoch 15: Loss: 0.154
Test accuracy is: 0.03710691823899371
Epoch 16: Loss: 0.127
Test accuracy is: 0.04

KeyboardInterrupt: ignored